In [ ]:
# gwo_sensor_placement.py
import random
import math
random.seed(42)

# ------------------ PROBLEM SETUP ------------------
# We assume 30 sensors, choose K=5 cluster heads (indices)
NUM_SENSORS = 30
K = 5

# Synthetic fitness function (lower is better)
# Mimics energy consumption + distances etc.
def fitness(solution):
    # sorted to avoid permutations being different solutions
    sol = sorted(solution)
    val = 0
    for i, idx in enumerate(sol):
        val += abs(idx - (i + 5)) * 1.7   # pattern creates unique optimum
    noise = random.uniform(0, 0.5)
    return val + noise

# ------------------ GWO PARAMETERS ------------------
POP_SIZE = 25
MAX_ITER = 40

# Wolves represented as real values in [0, NUM_SENSORS)
def random_wolf():
    return [random.uniform(0, NUM_SENSORS - 1) for _ in range(K)]

def discretize(wolf):
    # convert continuous wolf → integer unique cluster heads
    ints = sorted(list(set([int(round(x)) for x in wolf])))
    # pad if duplicates removed
    while len(ints) < K:
        r = random.randint(0, NUM_SENSORS - 1)
        if r not in ints:
            ints.append(r)
    return sorted(ints)

# ------------------ INITIALIZATION ------------------
wolves = [random_wolf() for _ in range(POP_SIZE)]
fitness_vals = [fitness(discretize(w)) for w in wolves]

# Identify α, β, δ wolves
def update_leaders():
    global alpha, beta, delta
    idx_sorted = sorted(range(POP_SIZE), key=lambda i: fitness_vals[i])
    alpha = wolves[idx_sorted[0]][:]
    beta  = wolves[idx_sorted[1]][:]
    delta = wolves[idx_sorted[2]][:]

update_leaders()

# ------------------ GWO MAIN LOOP ------------------
for t in range(MAX_ITER):
    a = 2 - 2 * (t / MAX_ITER)

    for i in range(POP_SIZE):
        Xi = wolves[i]

        new_pos = [0] * K
        for d in range(K):
            r1, r2 = random.random(), random.random()
            A1 = 2 * a * r1 - a
            C1 = 2 * r2

            D_alpha = abs(C1 * alpha[d] - Xi[d])
            X1 = alpha[d] - A1 * D_alpha

            r1, r2 = random.random(), random.random()
            A2 = 2 * a * r1 - a
            C2 = 2 * r2
            D_beta = abs(C2 * beta[d] - Xi[d])
            X2 = beta[d] - A2 * D_beta

            r1, r2 = random.random(), random.random()
            A3 = 2 * a * r1 - a
            C3 = 2 * r2
            D_delta = abs(C3 * delta[d] - Xi[d])
            X3 = delta[d] - A3 * D_delta

            new_pos[d] = (X1 + X2 + X3) / 3

        # clamp range
        for d in range(K):
            if new_pos[d] < 0: new_pos[d] = 0
            if new_pos[d] > NUM_SENSORS - 1: new_pos[d] = NUM_SENSORS - 1

        wolves[i] = new_pos
        fitness_vals[i] = fitness(discretize(new_pos))

    update_leaders()

# ------------------ OUTPUT ------------------
best_indices = discretize(alpha)
best_f = fitness_vals[sorted(range(POP_SIZE), key=lambda i: fitness_vals[i])[0]]

print("Application: Optimising the placement of sensors in a wireless sensor network")
print()
print("Output :")
print("Best Cluster Heads (indices):", best_indices)
print("Best Fitness Value :", round(best_f, 4))
